In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import tensorflow as tf
import tf_keras
import tensorflow_model_optimization as tfmot
from tf_keras import layers,losses,optimizers,Sequential
from tf_keras.models import Model
from tf_keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D,Input
from tf_keras.applications import VGG16


# Importing the dataset

In [3]:
from importing_dataset import load_dataset

In [4]:
train_examples,validation_examples,num_examples, num_classes, class_names = load_dataset('horses_or_humans',70)

# Reformating the dataset

In [5]:
from reformatting import reformat_image


In [6]:
# Create the base model from the pre-trained model VGG16 for transfer learning
vgg16_model = VGG16(input_shape=(224, 224, 3),
                    include_top=False,
                    weights='imagenet')
# Freeze the base model
vgg16_model.trainable = False

# Create a sequential mode
test_model = Sequential()
for layer in vgg16_model.layers[:-4]:
    test_model.add(layer)

del vgg16_model

for layer in test_model.layers:
    layer.trainable = False


# Add personal classification layers on top of the base model
max_pool_layer = layers.GlobalMaxPooling2D()
prediction_layer = layers.Dense(2, activation='softmax')  # For binary classification (2 classes)

inputs = Input(shape=(224, 224, 3))
x = test_model(inputs)  # Use default training=True behavior
x = max_pool_layer(x)
x = layers.Dropout(0.3)(x)  # Apply dropout during training
outputs = prediction_layer(x)
model = Model(inputs, outputs, name='model_confusion_matrix')

model.summary()

Model: "model_confusion_matrix"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 14, 14, 512)       7635264   
                                                                 
 global_max_pooling2d (Glob  (None, 512)               0         
 alMaxPooling2D)                                                 
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 7636290 (29.13 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 7635264 (29.13 MB)
___

In [11]:
from compilation import create_model_checkpoint
from ConfusionMatrix import ConfusionMatrixTest
from F1Score import F1Score

In [12]:
model.compile(
    optimizer = 'adam',
    loss = losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

In [13]:
from reformatting import reformat_image
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
class_names
train_batches = train_examples.cache().shuffle(num_examples//4).map(reformat_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(reformat_image).batch(BATCH_SIZE).prefetch(1)
EPOCHS = 3
history = model.fit(train_batches,
                    epochs = EPOCHS,
                    batch_size = BATCH_SIZE,
                    validation_data = validation_batches,callbacks=[F1Score(validation_data=validation_batches,labels=class_names)])

Epoch 1/3
1/1 [==============================] - 2s 2s/step

Confusion Matrix at epoch 1:
[[156   3]
 [ 30 119]]

Classification Report at epoch 1:
              precision    recall  f1-score   support

      horses     0.8387    0.9811    0.9043       159
      humans     0.9754    0.7987    0.8782       149

    accuracy                         0.8929       308
   macro avg     0.9071    0.8899    0.8913       308
weighted avg     0.9048    0.8929    0.8917       308

23/23 [==============================] - 162s 7s/step - loss: 190.4271 - accuracy: 0.7469 - val_loss: 33.8565 - val_accuracy: 0.8929
Epoch 2/3
1/1 [==============================] - 3s 3s/step

Confusion Matrix at epoch 2:
[[155   4]
 [  9 140]]

Classification Report at epoch 2:
              precision    recall  f1-score   support

      horses     0.9451    0.9748    0.9598       159
      humans     0.9722    0.9396    0.9556       149

    accuracy                         0.9578       308
   macro avg     0.9587   

In [14]:
class_names

['horses', 'humans']